In [1]:
#Importing Libraries
from keras.layers import Dense,Flatten
from keras.models import Model
from tensorflow.keras import layers
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [3]:
train_path = '../input/dermnet/train'
test_path = '../input/dermnet/test'

In [4]:
# add preprocessing layer to the front of Inception
incept = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

2022-04-05 12:55:20.184578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 12:55:20.264230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 12:55:20.264950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 12:55:20.266084: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

87924736/87910968 [==============================] - 0s 0us/step


In [5]:
# don't train existing weights
for layer in incept.layers:
  layer.trainable = False

In [6]:
#Getting Number of Categories
folders = glob('../input/dermnet/train/*')

In [7]:
x = Flatten()(incept.output)
x = layers.Dense(256, 'relu', kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
# create a model object
model = Model(inputs=incept.input, outputs=prediction)

In [10]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

data_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = data_datagen.flow_from_directory('../input/dermnet/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset="training")
valid_set = data_datagen.flow_from_directory(
    '../input/dermnet/train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical',
    subset="validation"
)

test_set = test_datagen.flow_from_directory('../input/dermnet/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [13]:
from keras.callbacks import ModelCheckpoint

In [14]:
filepath = './my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'

In [15]:
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

In [16]:
callbacks = [checkpoint]

In [17]:
#Fit the model
r = model.fit_generator(
  training_set,
  validation_data=valid_set,
  epochs=15,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set),
  callbacks=callbacks
)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-05 12:55:37.072685: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-04-05 12:55:42.459396: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


390/390 [==============================] - 321s 797ms/step - loss: 1.6148 - accuracy: 0.7642 - val_loss: 1.6384 - val_accuracy: 0.7758

Epoch 00001: val_loss improved from inf to 1.6384, saving model to ./my_best_model.epoch01-loss1.64.hdf5
Epoch 2/15
390/390 [==============================] - 240s 617ms/step - loss: 1.1868 - accuracy: 0.7730 - val_loss: 1.1491 - val_accuracy: 0.7997

Epoch 00002: val_loss improved from 1.6384 to 1.1491, saving model to ./my_best_model.epoch02-loss1.15.hdf5
Epoch 3/15
390/390 [==============================] - 244s 626ms/step - loss: 1.0109 - accuracy: 0.7961 - val_loss: 1.0102 - val_accuracy: 0.8046

Epoch 00003: val_loss improved from 1.1491 to 1.0102, saving model to ./my_best_model.epoch03-loss1.01.hdf5
Epoch 4/15
390/390 [==============================] - 244s 626ms/step - loss: 0.8917 - accuracy: 0.8317 - val_loss: 0.9819 - val_accuracy: 0.8245

Epoch 00004: val_loss improved from 1.0102 to 0.9819, saving model to ./my_best_model.epoch04-loss0.98

In [18]:
model.evaluate(test_set)

126/126 [==============================] - 53s 418ms/step - loss: 0.3904 - accuracy: 0.9696


[0.3903964042663574, 0.9695652186870575]